In [ ]:
!pip install pytorch_lightning
!pip install transformers
!pip install diffusers
!pip install datasets
!pip install huggingface huggingface_hub

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
import os
from PIL import Image
import numpy as np
import albumentations as A
import albumentations.pytorch as AP
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

from base64 import b64encode
import numpy as np
import json
import cv2
# from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
import subprocess

# For video display:
from IPython.display import HTML
from pathlib import Path
from torch import autocast
from torchvision import transforms as tfms
import pytorch_lightning as pl
from transformers import logging #CLIPTextModel, CLIPTokenizer, 
from transformers import BlipProcessor, BlipForConditionalGeneration

# torch.manual_seed(1)

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

from huggingface_hub import hf_hub_download
from diffusers import DiffusionPipeline
from huggingface_hub import notebook_login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from datasets import load_dataset

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface

In [ ]:
!huggingface_hub.login

/bin/bash: huggingface_hub.login: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%time
from zipfile import ZipFile
with ZipFile('/content/gdrive/My Drive/dataset.zip', mode='r') as input:
  input.extractall('/content')

CPU times: user 5.02 s, sys: 2.95 s, total: 7.97 s
Wall time: 26.2 s


In [ ]:
import os

In [ ]:
dir_path = '/content/dataset'
file_list = os.listdir(dir_path)
len(file_list)

10522

In [ ]:
file_list[0:5], file_list[-5:]

(['n03967562_36229.JPEG',
  'n07614500_41721.JPEG',
  'n07714990_6382.JPEG',
  'n04442312_5624.JPEG',
  'n03924679_2922.JPEG'],
 ['n09428293_33423.JPEG',
  'n04335435_4979.JPEG',
  'n07730033_8984.JPEG',
  'n04404412_15289.JPEG',
  'n04141975_23344.JPEG'])

In [ ]:
source_folder = "dataset"
destination_folder = "source"

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Get the list of image files in the source folder
image_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

# Iterate over each image file
for image_file in image_files:
    # Read the image
    image_path = os.path.join(source_folder, image_file)
    image = cv2.imread(image_path)
    
    # Apply Canny edge detection
    edges = cv2.Canny(image, 100, 200)  # Adjust the threshold values as needed
    
    # Save the edge image to the destination folder
    destination_path = os.path.join(destination_folder, image_file)
    cv2.imwrite(destination_path, edges)

    print(f"Processed image: {image_file}")

print("Edge detection completed.")

Streaming output truncated to the last 5000 lines.
Processed image: n04310018_13239.JPEG
Processed image: n04026417_26410.JPEG
Processed image: n07720875_2405.JPEG
Processed image: n04264628_15034.JPEG
Processed image: n04149813_14667.JPEG
Processed image: n04597913_2966.JPEG
Processed image: n04599235_18775.JPEG
Processed image: n04033901_32487.JPEG
Processed image: n07802026_10325.JPEG
Processed image: n11939491_12695.JPEG
Processed image: n04270147_18070.JPEG
Processed image: n03888257_40227.JPEG
Processed image: n04429376_23029.JPEG
Processed image: n04317175_8375.JPEG
Processed image: n03961711_5923.JPEG
Processed image: n04039381_19267.JPEG
Processed image: n07873807_6909.JPEG
Processed image: n04259630_3698.JPEG
Processed image: n09229709_6979.JPEG
Processed image: n04317175_3295.JPEG
Processed image: n07711569_1579.JPEG
Processed image: n09288635_13781.JPEG
Processed image: n04346328_3979.JPEG
Processed image: n03908714_11569.JPEG
Processed image: n04149813_8646.JPEG
Processed 

In [ ]:
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [ ]:
def generate_blip_caption(image_path, blip_model, blip_processor):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    inputs = blip_processor(image, return_tensors="pt").to("cuda")

    out = blip_model.generate(**inputs)
    decoded_caption = blip_processor.decode(out[0], skip_special_tokens=True)
    return decoded_caption


In [ ]:
def generate_prompt_json(source_folder, blip_model, prompt_file):
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    prompt_data = []

    # Iterate over the images in the source folder
    for i, filename in enumerate(os.listdir(source_folder)):
        print(str(i)+'/10000')
        # Check if it's an image file
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Compute Canny edges and get the destination path
            source_path = os.path.join(source_folder, filename)

            # Generate BLIP caption
            caption = generate_blip_caption(source_path, blip_model, blip_processor)

            # Create the prompt entry
            prompt_entry = {
                'source': filename,
                'target': filename,
                'prompt': caption
            }

            # Append to the prompt data list
            prompt_data.append(prompt_entry)

    # Write the prompt data to the JSON file
    with open(prompt_file, 'w') as f:
        json.dump(prompt_data, f, indent=4)

In [ ]:
source_folder = 'dataset'
prompt_file = 'prompt_forImages.json'

generate_prompt_json(source_folder, blip_model, prompt_file)


Streaming output truncated to the last 5000 lines.
5522/10000
5523/10000
5524/10000
5525/10000
5526/10000
5527/10000
5528/10000
5529/10000
5530/10000
5531/10000
5532/10000
5533/10000
5534/10000
5535/10000
5536/10000
5537/10000
5538/10000
5539/10000
5540/10000
5541/10000
5542/10000
5543/10000
5544/10000
5545/10000
5546/10000
5547/10000
5548/10000
5549/10000
5550/10000
5551/10000
5552/10000
5553/10000
5554/10000
5555/10000
5556/10000
5557/10000
5558/10000
5559/10000
5560/10000
5561/10000
5562/10000
5563/10000
5564/10000
5565/10000
5566/10000
5567/10000
5568/10000
5569/10000
5570/10000
5571/10000
5572/10000
5573/10000
5574/10000
5575/10000
5576/10000
5577/10000
5578/10000
5579/10000
5580/10000
5581/10000
5582/10000
5583/10000
5584/10000
5585/10000
5586/10000
5587/10000
5588/10000
5589/10000
5590/10000
5591/10000
5592/10000
5593/10000
5594/10000
5595/10000
5596/10000
5597/10000
5598/10000
5599/10000
5600/10000
5601/10000
5602/10000
5603/10000
5604/10000
5605/10000
5606/10000
5607/10000
560

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread('./source/' + source_filename)
        target = cv2.imread('./target/' + target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)


In [ ]:
import _locale
_locale._getdefaultlocale = (lambda *args: ['en_US', 'utf8'])

In [ ]:

import pytorch_lightning as pl
from torch.utils.data import DataLoader
# from dataset import MyDataset
# from cldm.logger import ImageLogger
# from cldm.model import create_model, load_state_dict
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
# Configs
resume_path = './models/checkpoint-epoch=01.ckpt'
batch_size = 10
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = True


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v21.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

In [ ]:
# Misc
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath='./models',
    filename='checkpoint-{epoch:02d}',
    save_last = True 
)

trainer = pl.Trainer(gpus=1, precision=16, callbacks=[logger, checkpoint_callback], \
 max_epochs=1, default_root_dir="./models")

In [ ]:
trainer.fit(model, dataloader)

# Save the model
model.save_checkpoint('saved_model.ckpt')